# Contrastive Learning for Predicting Cancer Prognosis Using Gene Expression Values

## Generate CPTAC-3 Datasets

*GenerateCPTAC3_Dataset.ipynb* notebook is offering detailed step-by-step instructions on how to generate CPTAC-3 validation dataset from matched CPTAC-3 data files.

In [1]:
import os.path

import pandas as pd
df_org = pd.read_csv('CPTAC-3&DKFZ/Matched_GeneExp.csv')
df_org

C:\Users\anchen.sun\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,bar,PFI,PFItime,gen_id,type,?|100130426,?|100133144,?|100134869,?|10357,?|10431,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,000ead0d-abf5-4606-be04-1ea31b999840,1.0,1816.0,KIRC,Tumor free,0.5915,3.6510,3.6510,0.0724,0.8032,...,1.2917,1.3374,3.3717,3.1189,12.1263,11.2384,19.4542,5.7679,11.8458,13.0883
1,a1da2f64-5a24-4e6b-abc1-e70aae23082c,1.0,6.0,NaN,Tumor free,0.4182,8.1859,8.1859,0.5531,0.7329,...,3.3659,4.5507,4.1481,4.3419,9.0435,0.0125,16.8398,25.8328,14.5501,13.1862
2,d8ef8a50-9276-4733-a15c-ad0fdc0b2910,1.0,1697.0,LUSC,Tumor free,0.6102,4.1890,4.1890,0.0560,1.0201,...,1.3741,1.3346,2.6259,3.4351,7.8449,0.0152,8.2575,28.7278,9.5081,5.6744
3,211bc56d-ce9d-43ab-add5-850b13c35f50,1.0,1745.0,NaN,Tumor free,0.0555,14.1692,14.1692,0.9782,2.1231,...,6.0434,6.0359,4.4431,9.9480,7.1779,0.9029,12.4283,14.4363,6.6687,16.1776
4,a59a90d9-f1b0-49dd-9c97-bcaa6ba55d44,1.0,1577.0,NaN,Tumor free,0.2864,6.6492,6.6492,0.0701,1.0785,...,4.9611,11.9825,0.9002,1.2866,6.4122,1.2924,5.4850,11.5992,7.4487,10.3251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,5cb7b327-5f17-44a8-aed3-989a1a99e780,0.0,-8.0,KIRC,NaN,0.1318,6.8913,6.8913,0.2906,1.8137,...,2.8320,7.1144,6.4632,4.0346,5.8547,0.0197,11.4827,30.1338,9.5814,16.9404
2188,1827eb4b-38f3-45b5-a833-156ba748c784,0.0,116.0,NaN,NaN,0.3544,3.4630,3.4630,0.4464,1.2959,...,5.9700,11.9148,4.3145,4.3234,10.3920,0.0756,19.7002,25.9744,13.2197,15.0347
2189,21b7f29a-1195-4625-b8b8-51352733d5d9,1.0,75.0,NaN,Tumor free,0.5336,12.3004,12.3004,1.5684,2.9941,...,4.7807,4.8258,2.0311,5.3098,5.3079,1.8062,11.5780,13.0523,6.9531,14.2014
2190,da1fdb3c-3031-4072-8ca8-96f1729ac276,0.0,68.0,NaN,NaN,0.0923,5.6654,5.6654,0.7626,1.1594,...,3.5996,4.5029,2.5394,4.8938,5.5769,0.1171,11.1636,17.4697,9.5596,11.6200


In [2]:
test_df = df_org.iloc[:, 5:]
upper_quartile = test_df.quantile(0.75, axis=1)
upper_quartile

0       8.62300
1       8.56150
2       7.81790
3       8.47355
4       7.23695
         ...   
2187    8.92350
2188    9.95710
2189    8.03925
2190    8.51805
2191    8.78850
Name: 0.75, Length: 2192, dtype: float64

In [3]:
length_df = pd.read_csv('CPTAC-3&DKFZ/mart_export.txt')
df_new_sample = pd.read_csv('CPTAC-3&DKFZ/gene_dict_sample_new.csv')
geneid_dict = df_new_sample.set_index('GeneID')['ReplacementGeneID'].to_dict()
symbol_dict = df_new_sample.set_index('GeneID')['OfficialSymbol'].to_dict()

def remove_none_values(dictionary):
    # Create a new dictionary without None values
    new_dict = {str(key): value for key, value in dictionary.items() if not isinstance(value, float)}
    return new_dict
def remove_none_values_gene(dictionary):
    # Create a new dictionary without None values
    new_dict = {str(key): str(int(value)) for key, value in dictionary.items() if value > 0}
    return new_dict

def get_value_from_dict(dictionary, key):
    # Get the value from the dictionary if the key exists
    if key in dictionary:
        return dictionary[key]

    # Return the original value if the key is not in the dictionary
    return key

new_geneid_dict = remove_none_values_gene(geneid_dict)
new_symbol_dict = remove_none_values(symbol_dict)

In [4]:
# Read the mapping DataFrame from CSV
mapping_df = pd.read_csv('CPTAC-3&DKFZ/gene_dict_symbol_new.csv')

# Create a dictionary mapping from the mapping DataFrame using '_id' as the key and 'MappedValue' as the value
mapping_dict = mapping_df.set_index('query')['_id'].to_dict()

# Create a new column in the main DataFrame by mapping the values from the mapping DataFrame
length_df['MappedColumn'] = length_df['Gene stable ID'].map(mapping_dict)

# Initialize an empty dictionary
mapped_length_dict = {}

# Iterate over each row in the dataframe
for index, row in length_df.iterrows():
    mapped_column = row['MappedColumn']
    transcript_length = row['Transcript length (including UTRs and CDS)']
    ensembl_canonical = row['Ensembl Canonical']

    # Check if the mapped_column is already present in the dictionary
    if mapped_column in mapped_length_dict:
        # If the current transcript_length is greater than the existing value, update the dictionary
        if transcript_length > mapped_length_dict[mapped_column]:
            mapped_length_dict[mapped_column] = transcript_length
    else:
        # If the mapped_column is not present, add it to the dictionary
        mapped_length_dict[mapped_column] = transcript_length

    # If the ensembl_canonical value is 1, update the dictionary regardless of the existing value
    if ensembl_canonical == 1:
        mapped_length_dict[mapped_column] = transcript_length
mapped_length_dict

{'4558': 71,
 '4549': 954,
 '4577': 69,
 '4550': 1559,
 '4567': 75,
 '4535': 956,
 '4565': 69,
 '4572': 72,
 '4569': 68,
 '4536': 1042,
 '4578': 68,
 '4553': 69,
 '4570': 73,
 '4511': 66,
 '4579': 66,
 '4512': 1542,
 '113219467': 69,
 '4555': 68,
 '4513': 684,
 '4566': 70,
 '4509': 207,
 '4508': 681,
 '4514': 784,
 '4563': 68,
 '4537': 346,
 '4573': 65,
 '4539': 297,
 '4538': 1378,
 '4564': 69,
 '4575': 59,
 '4568': 71,
 '4540': 1812,
 '4541': 525,
 '4556': 69,
 '4519': 1141,
 '4576': 66,
 '4571': 68,
 nan: 1052,
 '100874287': 443,
 '100419554': 1191,
 '64593': 1145,
 '359996': 1164,
 '377997': 868,
 'ENSG00000278478': 279,
 '643034': 1267,
 '83867': 1046,
 '100101120': 1367,
 'ENSG00000224518': 131,
 '386687': 783,
 '378953': 720,
 'ENSG00000275866': 385,
 '84672': 491,
 '106480250': 336,
 '100128190': 1337,
 '359799': 1064,
 '252955': 512,
 '359796': 298,
 '246122': 1367,
 '100873963': 5792,
 '643001': 1267,
 '9081': 1353,
 '642796': 509,
 'ENSG00000273589': 297,
 '100874520': 355,
 

In [5]:
# Function to retrieve mapped value from dictionary
def get_mapped_value(name):
    last_name = name.split('|')[-1]
    return mapped_length_dict.get(last_name)

columns_to_normalize = df_org.columns[5:]
times_df = df_org[columns_to_normalize].copy()
# Iterate over columns
for column in times_df.columns:
    column_name, length_key = column.split('|')
    new_id = get_value_from_dict(new_geneid_dict, length_key)
    try:
        length_value = mapped_length_dict[new_id]

        # Apply length value to each value in the column
        times_df[column] = times_df[column].apply(lambda x: x * length_value)
    except:
        print(times_df[column])
times_df

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2187   NaN
2188   NaN
2189   NaN
2190   NaN
2191   NaN
Name: ?|155060, Length: 2192, dtype: float64
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2187   NaN
2188   NaN
2189   NaN
2190   NaN
2191   NaN
Name: ?|317712, Length: 2192, dtype: float64
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2187   NaN
2188   NaN
2189   NaN
2190   NaN
2191   NaN
Name: ?|391343, Length: 2192, dtype: float64
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2187   NaN
2188   NaN
2189   NaN
2190   NaN
2191   NaN
Name: ?|441362, Length: 2192, dtype: float64
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2187   NaN
2188   NaN
2189   NaN
2190   NaN
2191   NaN
Name: ?|728788, Length: 2192, dtype: float64
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2187   NaN
2188   NaN
2189   NaN
2190   NaN
2191   NaN
Name: AQP7P3|441432, Length: 2192, dtype: float64

,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,414.6415,2778.4110,2778.4110,152.764,2003.9840,46.110,NaN,506.7972,0.0,NaN,...,4643.6615,2622.6414,16956.2793,17051.0263,40950.5151,49954.6880,158415.5506,12850.8812,135823.9428,83922.1796
1,293.1582,6229.4699,6229.4699,1167.041,1828.5855,615.648,NaN,263.8272,0.0,NaN,...,12100.4105,8923.9227,20860.7949,23737.1673,30539.8995,55.5625,137126.4914,57555.4784,166831.4466,84549.9144
2,427.7502,3187.8290,3187.8290,118.160,2545.1495,0.000,NaN,71.2920,0.0,NaN,...,4939.8895,2617.1506,13205.6511,18779.6917,26492.2273,67.5640,67240.8225,64005.5384,109019.8746,36384.2528
3,38.9055,10782.7612,10782.7612,2064.002,5297.1345,0.000,NaN,3693.7524,0.0,NaN,...,21726.0230,11836.3999,22344.3499,54385.7160,24239.7683,4013.3905,101203.6469,32164.0764,76463.3142,103730.7712
4,200.7664,5060.0412,5060.0412,147.911,2690.8575,0.000,NaN,133.8324,0.0,NaN,...,17835.1545,23497.6825,4527.1058,7033.8422,21653.9994,5744.7180,44664.3550,25843.0176,85406.7942,66204.5412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,92.3918,5244.2793,5244.2793,613.166,4525.1815,30.846,NaN,924.0972,0.0,NaN,...,10181.0400,13951.3384,32503.4328,22057.1582,19771.3219,87.5665,93503.6261,67138.1064,109860.3324,108621.8448
2188,248.4344,2635.3430,2635.3430,941.904,3233.2705,0.000,NaN,1171.1856,0.0,NaN,...,21462.1500,23364.9228,21697.6205,23636.0278,35093.7840,336.0420,160418.7286,57870.9632,151577.0802,96402.4964
2189,374.0536,9360.6044,9360.6044,3309.324,7470.2795,0.000,NaN,6358.8876,0.0,NaN,...,17186.6165,9463.3938,10214.4019,29028.6766,17924.7783,8028.5590,94279.6540,29080.5244,79724.2446,91059.3768
2190,64.7023,4311.3694,4311.3694,1609.086,2892.7030,32.277,NaN,485.0508,0.0,NaN,...,12940.5620,8830.1869,12770.6426,26754.4046,18833.1913,520.5095,90905.1948,38922.4916,109610.3736,74507.4400


In [6]:
df_org[columns_to_normalize] = times_df
df_org.to_csv('CPTAC-3&DKFZ/Matched_GeneExp_FPKM.csv', index=False)

In [7]:
import pandas as pd
df = pd.read_csv('CPTAC-3&DKFZ/Matched_GeneExp_FPKM.csv')
df

C:\Users\anchen.sun\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,bar,PFI,PFItime,gen_id,type,?|100130426,?|100133144,?|100134869,?|10357,?|10431,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,000ead0d-abf5-4606-be04-1ea31b999840,1.0,1816.0,KIRC,Tumor free,414.6415,2778.4110,2778.4110,152.764,2003.9840,...,4643.6615,2622.6414,16956.2793,17051.0263,40950.5151,49954.6880,158415.5506,12850.8812,135823.9428,83922.1796
1,a1da2f64-5a24-4e6b-abc1-e70aae23082c,1.0,6.0,NaN,Tumor free,293.1582,6229.4699,6229.4699,1167.041,1828.5855,...,12100.4105,8923.9227,20860.7949,23737.1673,30539.8995,55.5625,137126.4914,57555.4784,166831.4466,84549.9144
2,d8ef8a50-9276-4733-a15c-ad0fdc0b2910,1.0,1697.0,LUSC,Tumor free,427.7502,3187.8290,3187.8290,118.160,2545.1495,...,4939.8895,2617.1506,13205.6511,18779.6917,26492.2273,67.5640,67240.8225,64005.5384,109019.8746,36384.2528
3,211bc56d-ce9d-43ab-add5-850b13c35f50,1.0,1745.0,NaN,Tumor free,38.9055,10782.7612,10782.7612,2064.002,5297.1345,...,21726.0230,11836.3999,22344.3499,54385.7160,24239.7683,4013.3905,101203.6469,32164.0764,76463.3142,103730.7712
4,a59a90d9-f1b0-49dd-9c97-bcaa6ba55d44,1.0,1577.0,NaN,Tumor free,200.7664,5060.0412,5060.0412,147.911,2690.8575,...,17835.1545,23497.6825,4527.1058,7033.8422,21653.9994,5744.7180,44664.3550,25843.0176,85406.7942,66204.5412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,5cb7b327-5f17-44a8-aed3-989a1a99e780,0.0,-8.0,KIRC,NaN,92.3918,5244.2793,5244.2793,613.166,4525.1815,...,10181.0400,13951.3384,32503.4328,22057.1582,19771.3219,87.5665,93503.6261,67138.1064,109860.3324,108621.8448
2188,1827eb4b-38f3-45b5-a833-156ba748c784,0.0,116.0,NaN,NaN,248.4344,2635.3430,2635.3430,941.904,3233.2705,...,21462.1500,23364.9228,21697.6205,23636.0278,35093.7840,336.0420,160418.7286,57870.9632,151577.0802,96402.4964
2189,21b7f29a-1195-4625-b8b8-51352733d5d9,1.0,75.0,NaN,Tumor free,374.0536,9360.6044,9360.6044,3309.324,7470.2795,...,17186.6165,9463.3938,10214.4019,29028.6766,17924.7783,8028.5590,94279.6540,29080.5244,79724.2446,91059.3768
2190,da1fdb3c-3031-4072-8ca8-96f1729ac276,0.0,68.0,NaN,NaN,64.7023,4311.3694,4311.3694,1609.086,2892.7030,...,12940.5620,8830.1869,12770.6426,26754.4046,18833.1913,520.5095,90905.1948,38922.4916,109610.3736,74507.4400


In [8]:
filtered_df = df.drop_duplicates(subset='bar', keep='last')
df = filtered_df

In [9]:
class_counts = df['gen_id'].value_counts()
# Print the class counts
print(class_counts)

LUAD    229
KIRC    220
LUSC    108
Name: gen_id, dtype: int64


In [10]:
def is_float(value):
    result = True
    try:
        value_f = float(value)
    except:
        result = False
    return result

cancer = 'LUSC'
normalized_df = df[df.gen_id == cancer].copy()
#normalized_df = normalized_df[df_org.type == 'Tumor free']
normalized_df = normalized_df.dropna(subset=['gen_id'])
#normalized_df.PFI = 1
normalized_df = normalized_df[normalized_df['PFItime'].apply(is_float)]

normalized_df['PFItime'] = normalized_df['PFItime'].astype(float)
normalized_df['PFItime'] = normalized_df['PFItime'].astype(int)
normalized_df = normalized_df[normalized_df.PFItime >= 0]
normalized_df

,bar,PFI,PFItime,gen_id,type,?|100130426,?|100133144,?|100134869,?|10357,?|10431,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
2,d8ef8a50-9276-4733-a15c-ad0fdc0b2910,1.0,1697,LUSC,Tumor free,427.7502,3187.8290,3187.8290,118.160,2545.1495,...,4939.8895,2617.1506,13205.6511,18779.6917,26492.2273,67.5640,67240.8225,64005.5384,109019.8746,36384.2528
19,8f7bf749-60a0-4007-a75b-212e065d3e95,1.0,1435,LUSC,Tumor free,683.3348,6137.7694,6137.7694,477.282,3217.3025,...,6556.5610,9405.5443,10861.6342,14644.9996,31552.3241,4461.4465,54960.3642,89376.4428,112229.2080,52787.4312
25,d3c4d18f-aab6-401d-a27a-8a92dd9a4660,0.0,1522,LUSC,NaN,344.4013,8234.5527,8234.5527,2121.816,4138.7060,...,34289.8290,84299.0758,8331.0414,10130.3510,39975.9129,19729.5770,104375.3454,29541.4976,100512.1026,106125.0120
51,bc3c604c-c599-4749-b809-6e602a73e247,1.0,1017,LUSC,Tumor free,398.0979,3568.1007,3568.1007,508.088,2412.9145,...,5605.6835,3360.3696,16213.9989,17242.9180,33076.7019,144.9070,110770.8576,73850.6248,158864.8698,73441.7656
70,c6e1b9d8-09c8-4e05-ba87-ca985db813e3,0.0,92,LUSC,NaN,237.3586,1830.8138,1830.8138,787.452,4305.6215,...,22887.5675,48823.0170,5340.7980,14884.4542,22073.0851,247.1420,59290.8116,31568.5320,70205.1714,66271.2260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,2171413a-4d2c-4ad5-b539-f231a3d0a2de,1.0,1421,LUSC,Tumor free,124.5677,2688.2325,2688.2325,661.274,2556.1275,...,6794.9095,3227.2177,13374.1226,21839.5716,28965.2044,377.3805,85521.0432,55783.7728,116948.6136,52992.6152
2170,17b9c881-5f0c-44e8-8b59-2dfcab443513,0.0,1387,LUSC,NaN,538.5082,13746.7801,13746.7801,992.544,2848.2920,...,5476.6230,4615.6057,12511.1462,17261.5058,28122.9806,680.0850,75469.3240,63537.2128,143217.2196,53603.0376
2174,1c2653e9-8ba9-40da-a0f6-f5f06508d26a,1.0,1179,LUSC,Tumor free,284.4658,3617.4896,3617.4896,1038.120,2591.3070,...,5438.5160,3637.2628,17146.3755,15672.2489,40092.7571,700.5320,106017.7885,58971.3724,138583.8090,76068.7620
2184,df705995-5310-43be-a3ef-d97ab439bc86,1.0,540,LUSC,NaN,387.7231,4453.6764,4453.6764,584.681,2979.5290,...,8306.6070,25356.5144,14782.2426,19223.6121,66814.9581,7509.8275,57571.0100,15744.6076,67331.7918,78830.4104


In [11]:
import numpy as np
given_value = 1000  # From TCGA

# Select columns from column 6 to the end
columns_to_normalize = df.columns[5:]
df_to_norm = df.iloc[:, 5:]
upper_quartile = df_to_norm.quantile(0.75, axis=1)
upper_quartile_rate = upper_quartile / given_value

# Normalize selected columns
normalized_df = df.copy()
normalized_df[columns_to_normalize] = normalized_df[columns_to_normalize].div(upper_quartile_rate, axis=0)
normalized_df[columns_to_normalize] = normalized_df[columns_to_normalize].applymap(lambda x: np.log2(x + 1))
normalized_df

,bar,PFI,PFItime,gen_id,type,?|100130426,?|100133144,?|100134869,?|10357,?|10431,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
1,a1da2f64-5a24-4e6b-abc1-e70aae23082c,1.0,6.0,NaN,Tumor free,3.302168,7.564949,7.564949,5.181372,5.814789,...,8.519123,8.081216,9.303209,9.489285,9.852386,1.422230,12.017912,10.765911,12.300734,11.320496
16,1256a2fe-44c4-4019-a5bc-d188e3c76079,1.0,344.0,KIRC,Tumor free,3.504921,5.897367,5.897367,3.637749,5.668054,...,7.454454,8.099515,9.561023,9.080178,9.957654,6.591211,11.803737,9.516425,11.704057,11.374023
21,2d75c2fc-90f0-4169-92ff-4ac8e4817ac7,1.0,1040.0,NaN,Tumor free,2.558087,6.795021,6.795021,6.231898,5.988487,...,8.004195,7.422800,9.276715,9.289171,10.103149,4.132799,11.685192,10.928169,12.464644,11.216644
25,d3c4d18f-aab6-401d-a27a-8a92dd9a4660,0.0,1522.0,LUSC,NaN,3.309407,7.742287,7.742287,5.805178,6.756429,...,9.795177,11.091949,7.759016,8.039948,10.016296,8.998954,11.400014,9.580400,11.345623,11.423989
35,46fd5848-ddb3-4cbf-97ce-c94d72910679,0.0,399.0,NaN,NaN,2.465440,7.369721,7.369721,5.087934,6.585367,...,8.775004,8.797697,9.128043,9.274606,9.442500,4.592617,11.333047,9.917401,11.508326,11.293919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,5cb7b327-5f17-44a8-aed3-989a1a99e780,0.0,-8.0,KIRC,NaN,1.887941,7.269744,7.269744,4.242287,7.058460,...,8.222273,8.675485,9.893658,9.335031,9.177452,1.831877,11.417099,10.939415,11.649596,11.633245
2188,1827eb4b-38f3-45b5-a833-156ba748c784,0.0,116.0,NaN,NaN,2.888338,6.107148,6.107148,4.659954,6.398276,...,9.114403,9.236741,9.130117,9.253357,9.822813,3.272401,12.014123,10.543810,11.932352,11.279654
2189,21b7f29a-1195-4625-b8b8-51352733d5d9,1.0,75.0,NaN,Tumor free,3.705789,8.240818,8.240818,6.749448,7.916585,...,9.115257,8.256523,8.366351,9.870389,9.175820,8.020149,11.568789,9.872960,11.326948,11.518667
2190,da1fdb3c-3031-4072-8ca8-96f1729ac276,0.0,68.0,NaN,NaN,1.571866,7.049429,7.049429,5.645700,6.479033,...,8.627830,8.078142,8.608809,9.673822,9.168066,4.076497,11.437161,10.214097,11.707022,11.150297


In [12]:
test_df = normalized_df.iloc[:, 5:]
upper_quartile = test_df.quantile(0.75, axis=1)
upper_quartile

1       9.967226
16      9.967226
21      9.967226
25      9.967226
35      9.967226
          ...   
2187    9.967226
2188    9.967226
2189    9.967226
2190    9.967226
2191    9.967226
Name: 0.75, Length: 1168, dtype: float64

In [13]:
#LUSC
#Norm by VCP, RAB7A, without log all: 32779.5704048583, without log risk: 33751.31946502057
#Norm by all 10: without log all: 54967.24772672064, without log risk: 55947.00704526749
#LUAD
#Norm by VCP, RAB7A, without log all: 20221.128207171314, without log risk: 20835.954516129033
#Norm by all 10: without log all: 42871.3474442231, without log risk: 43978.18987096774
import os
import numpy as np

def is_float(value):
    result = True
    try:
        value_f = float(value)
    except:
        result = False
    return result

cancer_list = ['LUSC', 'LUAD']
#house_keeper_list_list = [['C1orf43', 'CHMP2A', 'GPI', 'PSMB2', 'PSMB4', 'RAB7A', 'REEP5', 'SNRPD3', 'VCP', 'VPS29'], ['VCP', 'RAB7A', 'GPI']]
house_keeper_list_list = [['VPS29', 'CHMP2A', 'SNRPD3', 'PSMB2']]
#house_keeper_list = ['C1orf43', 'CHMP2A', 'EMC7', 'GPI', 'PSMB2', 'PSMB4', 'RAB7A', 'REEP5', 'SNRPD3', 'VCP', 'VPS29']
#house_keeper_list = ['VCP', 'RAB7A', 'GPI']

#TCGA_3genes_mean = 21419.06700584365  # From TCGA
#TCGA_10genes_mean = 43416.288172294895  # From TCGA
#TCGA_genes_mean_list = [[54967.24772672064, 42871.3474442231, 45076.818913043484], [32779.5704048583, 20221.128207171314, 23517.183516068053]]
TCGA_genes_mean_list = [[8192.112344129555, 7384.9791414342635, 6838.665030245747]]
norm_fill_method = ['FZ', 'FM']

cancer_i = -1
for cancer in cancer_list:
    cancer_i += 1
    for k in range(len(house_keeper_list_list)):
        for norm_mth in norm_fill_method:

            if not os.path.exists('CPTAC-3&DKFZ/Validation/CPTAC3_{}Genes_Log_Norm_{}'.format(len(house_keeper_list), norm_mth)):
                os.makedirs('CPTAC-3&DKFZ/Validation/CPTAC3_{}Genes_Log_Norm_{}'.format(len(house_keeper_list), norm_mth))

            house_keeper_list = house_keeper_list_list[k]
            TCGA_genes_mean = TCGA_genes_mean_list[k]
            print(cancer, len(house_keeper_list))
            # Select columns from column 6 to the end
            normalized_df = df[df.gen_id == cancer].copy()
            #normalized_df = normalized_df[df.type == 'Tumor free']
            normalized_df = normalized_df.dropna(subset=['gen_id'])
            #normalized_df.PFI = 1
            normalized_df = normalized_df[normalized_df['PFItime'].apply(is_float)]

            normalized_df['PFItime'] = normalized_df['PFItime'].astype(float)
            normalized_df['PFItime'] = normalized_df['PFItime'].astype(int)
            normalized_df = normalized_df[normalized_df.PFItime >= 0]

            columns_to_normalize = normalized_df.columns[5:].copy()

            value_df = normalized_df.iloc[:, 5:].mean()

            total_value = 0.0
            for index, row in value_df.to_frame().iterrows():
                # Access row elements using row['column_name'] or row[index]
                gene_name = index.split('|')[0]
                if gene_name in house_keeper_list:
                    value = row[0]
                    total_value += value

            print(total_value)

            norm_rate = total_value / TCGA_genes_mean[cancer_i]

            normalized_df[columns_to_normalize] = normalized_df[columns_to_normalize].div(norm_rate, axis=0)
            normalized_df[columns_to_normalize] = normalized_df[columns_to_normalize].applymap(lambda x: np.log2(x + 1))

            df_risk = normalized_df

            if norm_mth == 'FZ':
                df_risk = df_risk.fillna(0)
            else:
                # Calculate the mean of each row
                row_means = df_risk[columns_to_normalize].mean(axis=1)

                # Apply the fillna method to each row, replacing NaN values with the corresponding row mean
                df_risk[columns_to_normalize] = df_risk[columns_to_normalize].apply(lambda row: row.fillna(row_means[row.name]), axis=1)

            #df_risk = df_risk[df_risk['PFItime'].apply(is_float)]

            #df_risk['PFItime'] = df_risk['PFItime'].astype(float)
            #df_risk['PFItime'] = df_risk['PFItime'].astype(int)
            #df_risk = df_risk[df_risk.PFItime >= 0]

            data_get = df_risk
            n = 2
            div_point = [1] * n
            threshold = 3 * 365
            data_get = data_get[(data_get.PFI == 1) | (data_get.PFItime > threshold)]
            data_get.insert(4, 'predicted_label', 0, True)
            timelabel = []
            for item in data_get['PFItime'].tolist():
                i = 0
                if item < threshold:
                    timelabel.append(0)
                else:
                    timelabel.append(1)

            data_get['predicted_label'] = np.array(timelabel)

            if cancer == 'LUSC':
                df_SM = pd.read_csv('DataSet/CancerRNA_SM_WholeTimeSeq_6.txt')
                df_SM_columns = df_SM.columns
                sub_df = data_get.loc[:, df_SM_columns]
                sub_df.to_csv('CPTAC-3&DKFZ/Validation/CPTAC3_{}Genes_Log_Norm_{}/CancerRNA_CPTAC3_{}_Risk_{}.txt'.format(len(house_keeper_list), norm_mth, cancer, n), index=None)
            else:
                data_get.to_csv('CPTAC-3&DKFZ/Validation/CPTAC3_{}Genes_Log_Norm_{}/CancerRNA_CPTAC3_{}_Risk_{}.txt'.format(len(house_keeper_list), norm_mth, cancer, n), index=None)

            data_get = df_risk
            timelist = data_get['PFItime'].tolist()
            timelist.sort()
            n = 3
            div_point = []
            timelabel = []
            for i in range(n):
                div_point.append(timelist[int(len(timelist) / n * i)])
            print(div_point)
            for item in data_get['PFItime'].tolist():
                i = 0
                while item >= div_point[i] and i < (n - 1):
                    i += 1
                if item >= div_point[i] and i == (n - 1):
                    i += 1
                timelabel.append(i - 1)
            data_get['predicted_label'] = np.array(timelabel)

            if cancer == 'LUSC':
                df_SM = pd.read_csv('DataSet/CancerRNA_SM_WholeTimeSeq_6.txt')
                df_SM_columns = df_SM.columns
                sub_df = data_get.loc[:, df_SM_columns]
                sub_df.to_csv('CPTAC-3&DKFZ/Validation/CPTAC3_{}Genes_Log_Norm_{}/CancerRNA_CPTAC3_{}_WholeTimeSeq_{}.txt'.format(len(house_keeper_list), norm_mth, cancer, n), index=None)
            else:
                data_get.to_csv('CPTAC-3&DKFZ/Validation/CPTAC3_{}Genes_Log_Norm_{}/CancerRNA_CPTAC3_{}_WholeTimeSeq_{}.txt'.format(len(house_keeper_list), norm_mth, cancer, n), index=None)
            print('Finish {} {} {}.'.format(cancer, len(house_keeper_list), norm_mth))

LUSC 4
220876.42259117647


C:\Users\anchen.sun\AppData\Local\Temp\ipykernel_15684\2128805164.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_get['predicted_label'] = np.array(timelabel)


[3, 540, 1412]
Finish LUSC 4 FZ.
LUSC 4
220876.42259117647


C:\Users\anchen.sun\AppData\Local\Temp\ipykernel_15684\2128805164.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_get['predicted_label'] = np.array(timelabel)


[3, 540, 1412]
Finish LUSC 4 FM.
LUAD 4
212482.02562572816


C:\Users\anchen.sun\AppData\Local\Temp\ipykernel_15684\2128805164.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_get['predicted_label'] = np.array(timelabel)


[1, 403, 889]
Finish LUAD 4 FZ.
LUAD 4
212482.02562572816


C:\Users\anchen.sun\AppData\Local\Temp\ipykernel_15684\2128805164.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_get['predicted_label'] = np.array(timelabel)


[1, 403, 889]
Finish LUAD 4 FM.


In [14]:
df_risk

,bar,PFI,PFItime,gen_id,type,?|100130426,?|100133144,?|100134869,?|10357,?|10431,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
2,d8ef8a50-9276-4733-a15c-ad0fdc0b2910,1,1697.0,LUSC,Tumor free,4.080533,6.902480,6.902480,2.432231,6.580698,...,7.530101,6.620530,8.943812,9.450960,9.946756,1.813264,11.289638,11.218526,11.986599,10.404103
19,8f7bf749-60a0-4007-a75b-212e065d3e95,1,1435.0,LUSC,Tumor free,4.724105,7.841812,7.841812,4.229730,6.915647,...,7.936636,8.455423,8.662530,9.092777,10.198698,7.383970,10.998821,11.700052,12.028445,10.940652
25,d3c4d18f-aab6-401d-a27a-8a92dd9a4660,1,1522.0,LUSC,0,3.788346,8.264186,8.264186,6.321251,7.276312,...,10.318635,11.615700,8.280938,8.562230,10.539824,9.522047,11.923804,10.103778,11.869407,11.947782
51,bc3c604c-c599-4749-b809-6e602a73e247,1,1017.0,LUSC,Tumor free,3.983226,7.063776,7.063776,4.315296,6.504550,...,7.711585,6.977906,9.239353,9.327974,10.266711,2.676415,12.009580,11.424859,12.529696,11.416852
70,c6e1b9d8-09c8-4e05-ba87-ca985db813e3,1,92.0,LUSC,0,3.297661,6.111302,6.111302,4.921469,7.332993,...,9.735981,10.828079,7.642091,9.116132,9.683767,3.350113,11.108186,10.199438,11.351853,11.268691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,0449aa40-ee66-45c5-baa7-db48dbcc7702,1,727.0,LUSC,Tumor free,4.475955,7.374421,7.374421,4.430627,6.641535,...,7.439605,6.750299,8.875952,8.957385,10.617158,3.230623,11.831033,11.525540,12.386683,11.159754
2087,f05dc58e-c9dc-48fc-bf07-e0aca961797a,1,505.0,LUSC,Tumor free,3.940476,9.197858,9.197858,2.653618,6.631842,...,9.455163,10.155428,8.418999,8.912284,10.744436,7.904683,11.010693,11.352453,12.496928,10.767907
2088,f1aadf48-b132-48fd-9106-4a01670114e0,1,1462.0,LUSC,Tumor free,2.818623,6.410318,6.410318,4.149019,6.158022,...,7.447367,7.458300,8.655784,8.885227,10.342677,3.111423,11.947343,11.568533,12.375176,10.975914
2110,c64ccd3d-14a0-45e1-920c-a6babdcb82af,1,442.0,LUSC,0,2.717537,6.982298,6.982298,4.845487,6.678752,...,7.624184,6.858142,9.388729,9.216644,10.607534,4.217274,11.800276,11.131505,12.495500,11.363028


In [15]:
df_risk = normalized_df[df.type == 'Tumor free']
df_risk = df_risk.dropna(subset=['gen_id'])
df_risk.PFI = 1
df_risk = df_risk.fillna(0)
df_risk['PFItime'] = df_risk['PFItime'].astype(float)
df_risk['PFItime'] = df_risk['PFItime'].astype(int)
df_risk = df_risk[df_risk.PFItime > 0]
df_risk

,bar,PFI,PFItime,gen_id,type,?|100130426,?|100133144,?|100134869,?|10357,?|10431,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
16,1256a2fe-44c4-4019-a5bc-d188e3c76079,1,344,KIRC,Tumor free,3.504921,5.897367,5.897367,3.637749,5.668054,...,7.454454,8.099515,9.561023,9.080178,9.957654,6.591211,11.803737,9.516425,11.704057,11.374023
56,c6953ff1-355b-4b83-8e9d-4dae4a293bf1,1,345,KIRC,Tumor free,4.871591,6.196225,6.196225,1.911402,5.724316,...,6.766202,7.981755,9.768835,9.143722,9.607319,8.466735,11.845732,8.883503,11.560490,11.194695
63,d497bd22-62ba-4a9c-8f0c-28aae3838c61,1,1591,KIRC,Tumor free,4.527125,5.803198,5.803198,4.666394,5.872758,...,7.560112,8.187764,9.557756,8.797288,9.524991,3.899716,11.711797,9.171631,11.255634,11.247131
107,d999afdf-cb75-4da0-8648-ad3ec6905918,1,1086,KIRC,Tumor free,2.337395,8.702739,8.702739,5.586686,7.266870,...,6.533044,7.305106,9.628263,8.382372,9.697212,5.151528,10.266394,11.187587,12.940595,10.898862
111,82ac2853-17a0-446c-90b0-307a72c961d1,1,1210,KIRC,Tumor free,1.543996,5.635711,5.635711,5.402650,5.886568,...,7.490635,7.894971,10.021895,9.048598,9.981687,6.705715,11.885449,8.969497,11.556657,11.327729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163,01b5cff1-824f-4fd4-a149-7bcbacf8c4dc,1,352,KIRC,Tumor free,3.794283,6.417834,6.417834,4.482619,6.586354,...,6.989401,6.917168,9.069581,8.722240,10.431023,9.415324,12.058009,8.527868,11.973381,11.334908
2167,2171413a-4d2c-4ad5-b539-f231a3d0a2de,1,1421,LUSC,Tumor free,2.303377,6.425314,6.425314,4.452535,6.353484,...,7.752930,6.686141,8.726554,9.432733,9.839596,3.692042,11.400513,10.784363,11.851893,10.710352
2174,1c2653e9-8ba9-40da-a0f6-f5f06508d26a,1,1179,LUSC,Tumor free,3.186210,6.700904,6.700904,4.933939,6.225076,...,7.284477,6.708693,8.934763,8.805349,10.158513,4.389056,11.560620,10.714782,11.946960,11.081878
2178,db1f6497-d75d-4911-b1b3-32f97632ca3e,1,1475,LUAD,Tumor free,3.254470,6.855145,6.855145,3.539345,6.350909,...,7.398811,7.131214,8.302200,8.796740,9.976082,3.014348,11.556194,11.061264,12.037517,10.769950


In [16]:
# Count the number of samples for each class in the "gen_id" column
class_counts = df_risk['gen_id'].value_counts()
# Print the class counts
print(class_counts)

LUAD    127
KIRC    124
LUSC     65
Name: gen_id, dtype: int64


In [17]:
import numpy as np
cancer_type_list = ['LUAD', 'LUSC', 'KIRC']
for cancer_get in cancer_type_list:
    data_get = df_risk[df_risk.gen_id == cancer_get]
    n = 2
    div_point = [1] * n
    threshold = 3 * 365
    data_get = data_get[(data_get.PFI == 1) | (data_get.PFItime > threshold)]
    data_get.insert(4, 'predicted_label', 0, True)
    timelabel = []
    for item in data_get['PFItime'].tolist():
        i = 0
        if item < threshold:
            timelabel.append(0)
        else:
            timelabel.append(1)

    data_get['predicted_label'] = np.array(timelabel)
    data_get.to_csv('DataSet/CancerRNA_CPTAC3_{}_Risk_{}.txt'.format(cancer_get, n), index=None)

In [18]:
cancer_type_list = ['LUAD', 'LUSC']
for cancer_get in cancer_type_list:
    data_get = df_risk[df_risk.gen_id == cancer_get]
    timelist = data_get['PFItime'].tolist()
    timelist.sort()
    n = 3
    div_point = []
    timelabel = []
    for i in range(n):
        div_point.append(timelist[int(len(timelist) / n * i)])
    print(div_point)
    for item in data_get['PFItime'].tolist():
        i = 0
        while item >= div_point[i] and i < (n - 1):
            i += 1
        if item >= div_point[i] and i == (n - 1):
            i += 1
        timelabel.append(i - 1)
    data_get['predicted_label'] = np.array(timelabel)
    data_get.to_csv('SingleCancerDataSet/CancerRNA_CPTAC3_{}_WholeTimeSeq_{}.txt'.format(cancer_get, n), index=None)

[1, 418, 1076]


C:\Users\anchen.sun\AppData\Local\Temp\ipykernel_28444\1494050577.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_get['predicted_label'] = np.array(timelabel)


[3, 727, 1427]


C:\Users\anchen.sun\AppData\Local\Temp\ipykernel_28444\1494050577.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_get['predicted_label'] = np.array(timelabel)


In [19]:
df_SM = pd.read_csv('DataSet/CancerRNA_SM_WholeTimeSeq_6.txt')
df_SM

,bar,PFI,PFItime,gen_id,predicted_label,type,?|100133144,?|100134869,?|10357,?|10431,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,TCGA-2F-A9KO,1,651,BLCA,3,TCGA-2F-A9KO-01A-11R-A38B-07,4.422051,5.280648,6.955243,9.457465,...,9.442647,9.117692,6.695214,9.603025,10.503557,2.772076,9.449064,12.057785,10.804744,9.163889
1,TCGA-2F-A9KP,1,343,BLCA,1,TCGA-2F-A9KP-01A-11R-A38B-07,4.099144,3.762976,7.117757,9.786647,...,9.490087,9.922764,5.249332,8.311467,10.473422,4.174886,10.576891,11.600977,10.520373,9.750859
2,TCGA-2F-A9KQ,0,2886,BLCA,4,TCGA-2F-A9KQ-01A-11R-A38B-07,3.841299,3.557557,6.723531,9.899513,...,9.059442,9.636780,5.937083,8.567207,9.772207,1.513188,10.618532,11.677887,9.553783,9.281311
3,TCGA-2F-A9KR,1,3086,BLCA,4,TCGA-2F-A9KR-01A-11R-A38B-07,4.031422,5.445627,7.204111,8.931358,...,9.271108,9.153329,5.541852,8.879935,10.910755,1.535804,9.550477,12.124968,9.832555,9.871106
4,TCGA-2F-A9KT,0,3314,BLCA,4,TCGA-2F-A9KT-01A-11R-A38B-07,3.908198,4.220539,7.169544,9.900530,...,9.507051,10.087529,4.211324,8.142184,9.875570,2.960419,9.780477,11.752552,9.733351,9.303473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,TCGA-O2-A52S,1,246,LUSC,1,TCGA-O2-A52S-01A-11R-A262-07,4.388899,5.588043,7.473446,10.215933,...,10.070658,10.994877,6.053055,8.989962,10.444808,6.991431,9.183134,11.389475,10.064123,9.614918
1801,TCGA-O2-A52V,1,690,LUSC,3,TCGA-O2-A52V-01A-31R-A262-07,4.958249,4.026419,7.567736,10.286592,...,9.281594,10.389158,5.753463,8.522954,10.374616,2.868529,9.726026,12.573491,9.377070,9.606688
1802,TCGA-O2-A52W,0,261,LUSC,1,TCGA-O2-A52W-01A-11R-A26W-07,5.772395,5.103947,8.029746,9.626368,...,9.227864,10.038988,5.527852,8.400521,10.968444,6.680296,9.263619,11.323049,9.514227,9.575456
1803,TCGA-O2-A5IB,1,253,LUSC,1,TCGA-O2-A5IB-01A-11R-A27Q-07,6.200979,5.432398,7.332368,8.851321,...,8.747931,10.596059,6.528581,9.192618,11.019278,7.242469,10.175200,10.211937,9.704479,10.434753


In [20]:
df_SM_columns = df_SM.columns

dataget = pd.read_csv('DataSet/CancerRNA_CPTAC3_LUSC_Risk_2.txt')
sub_df = dataget.loc[:, df_SM_columns]
sub_df.to_csv('DataSet/CancerRNA_CPTAC3_LUSCSM_Risk_2.txt', index=None)

dataget = pd.read_csv('SingleCancerDataSet/CancerRNA_CPTAC3_LUSC_WholeTimeSeq_3.txt')
sub_df = dataget.loc[:, df_SM_columns]
sub_df.to_csv('SingleCancerDataSet/CancerRNA_CPTAC3_LUSCSM_WholeTimeSeq_3.txt', index=None)